In [82]:
%logstop
%logstart -rtq ~/.logs/dw.py append
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [9]:
from static_grader import grader

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We'll work with the same medical data used in the `pw` miniproject but leverage the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [10]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

mkdir: cannot create directory ‘dw-data’: File exists
File ‘./dw-data/201701scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/201606scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/practices.csv.gz’ already there; not retrieving.

File ‘./dw-data/chem.csv.gz’ already there; not retrieving.



## Loading the data

Similar to the `PW` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a Pandas DataFrame by making use of the `gzip` package to decompress the files and Panda's `read_csv` methods to parse the data into a DataFrame. You may want to check the Pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [PW miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [11]:
import pandas as pd
import numpy as np
import gzip

In [12]:
# load the 2017 data
scripts = pd.read_csv('dw-data/201701scripts_sample.csv.gz')
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [13]:
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
practices = pd.read_csv('dw-data/practices.csv.gz', names= col_names)
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [14]:
chem = pd.read_csv('dw-data/chem.csv.gz')


In [15]:
chem.shape

(3487, 2)

Now that we've loaded in the data, let's first replicate our results from the `PW` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `PW` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `PW` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: [('act_cost', (total, mean, std, q25, median, q75)), ...]

In [16]:
stats = scripts.describe()

In [17]:
def compute_summary_stats(df):
    row_order = ['items', 'quantity', 'nic', 'act_cost']
    col_order = ['total', 'mean', 'std', '25%','50%','75%']
    total = df[row_order].sum()
    stats = (df.describe()
             .drop(['min', 'max', 'count'],axis =0)
             .T
             .assign(total=total)
             .loc[row_order, col_order])
               
    return stats


In [18]:
stats = compute_summary_stats(scripts)

In [19]:
summary_stats = [(row.Index,row[1:]) for row in stats.itertuples()]

In [20]:
grader.score.dw__summary_statistics(summary_stats)

Your score:  1.0


## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [21]:
bnf_name_total = scripts.groupby('bnf_name')['items'].sum()
bnf_name_total

bnf_name
365 Film 10cm x 12cm VP Adh Film Dress       2
365 Non Adherent 10cm x 10cm Pfa Plas Fa     3
365 Non Adherent 10cm x 20cm Pfa Plas Fa     1
365 Non Woven Island 8cm x 10cm Adh Dres     1
365 Transpt Island 5cm x 7.2cm VP Adh Fi     2
                                            ..
nSpire PiKo-1 Stnd Range Peak Flow Meter     1
nSpire Pocket Peak Low Range Peak Flow M     4
nSpire Pocket Peak Stnd Range Peak Flow      8
oraNurse_Toothpaste Orig (1450ppm)           4
palmdoc (Reagent)_Strips                    59
Name: items, Length: 13471, dtype: int64

In [22]:
max_items = bnf_name_total.max()

In [23]:
bnf_name_total[bnf_name_total == max_items]

bnf_name
Omeprazole_Cap E/C 20mg    218583
Name: items, dtype: int64

In [24]:
most_common_item = [(bnf_name_total.idxmax(),bnf_name_total.max())]

In [25]:
grader.score.dw__most_common_item(most_common_item)

Your score:  1.0


## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the PW miniproject).

In [26]:
practices_unique = (practices.sort_values('post_code')
                    .drop_duplicates(subset= ['code'], keep = 'first'))

In [27]:
scripts.merge( practices_unique, left_on ='practice', right_on ='code')

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,code,name,addr_1,addr_2,borough,village,post_code
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
5,N85639,0406000T0,Prochlpzine Mal_Tab 5mg,1,0.97,0.91,28,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
6,N85639,0407010F0,Co-Codamol_Cap 30mg/500mg,1,0.84,0.89,24,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
7,N85639,0407010F0,Zapain_Tab 30mg/500mg,1,3.03,2.82,100,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
8,N85639,0407010H0,Paracet_Oral Susp Paed 120mg/5ml,1,0.62,0.69,100,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF
9,N85639,0501011P0,Phenoxymethylpenicillin Pot_Tab 250mg,2,5.94,5.72,160,N85639,GP OOH VCH,VICTORIA CENTRAL HOSPITAL,MILL LANE,WALLASEY,NaN,CH44 5UF


In [28]:
scripts_with_post_code = scripts.merge( practices_unique, left_on ='practice', right_on ='code')

In [29]:
scripts_with_post_code.groupby(['post_code','bnf_name'])['items'].sum()

post_code  bnf_name                                
B11 4BW    3m Health Care_Cavilon Durable Barrier C     7
           3m Health Care_Cavilon No Sting Barrier      2
           Abasaglar KwikPen_100u/ml 3ml Pf Pen         2
           Abidec_Dps                                  63
           Able Spacer + Sml/Med Mask                   1
                                                       ..
YO16 4LZ   Zydol SR_Tab 50mg                            1
           easiGRIP 12cm x 1m Stkntte Elasctd Tublr     1
           iMEDicare SomaCorrect Xtra Vacuum Pump       1
           iMEDicare_Urine Collection Bag 500ml         1
           kliniderm superabsorbent 20cm x 30cm Pfa     1
Name: items, Length: 579197, dtype: int64

In [30]:
out_put = (scripts_with_post_code.groupby(['post_code','bnf_name'])['items'].sum().reset_index('bnf_name'))


In [31]:
totals = out_put.groupby('post_code')['items'].sum()

In [32]:
out_put.groupby('post_code')['items'].sum()

post_code
B11 4BW     22731
B12 9LP     17073
B18 7AL     20508
B21 9RY     31027
B23 6DJ     28011
            ...  
WS3 3JP     36890
WS9 8AJ     24483
WV13 2DR    36381
YO11 1UB    28507
YO16 4LZ    53847
Name: items, Length: 259, dtype: int64

In [33]:
ratio = out_put['items']/totals

In [34]:
out_put['ratio'] = ratio


In [35]:
out_put.sort_values('ratio',ascending=False).head()

,bnf_name,items,ratio
post_code,,,
TN24 0GP,Amoxicillin_Cap 500mg,1440,0.080767
N9 0TW,Influenza_Vac Inact 0.5ml Pfs,2384,0.070656
NG8 6AS,Coloplast_Brava Adh Remover Spy 50ml,332,0.061722
CH62 6EE,Influenza_Vac Inact 0.5ml Pfs,1842,0.057586
WS2 0BA,Influenza_Vac Inact 0.5ml Pfs,977,0.050525


In [36]:
#for post_code in out_put.index:
    #out_put.loc[post_code].sort_values('ratio',ascending=False).head(1)

In [37]:
#out_put.loc['B11 4BW'].sort_values('ratio',ascending=False).head()

In [38]:
#out_put.groupby('post_code')['ratio'].max()

In [39]:
def top_row(df):
    return df.sort_values('ratio', ascending = False).iloc[0]
result = out_put.groupby('post_code').apply(top_row)
result = result.reset_index().loc[:,['post_code', 'bnf_name', 'ratio']].head(100)
result.head()

,post_code,bnf_name,ratio
0,B11 4BW,Salbutamol_Inha 100mcg (200 D) CFF,0.031059
1,B12 9LP,Paracet_Tab 500mg,0.024893
2,B18 7AL,Salbutamol_Inha 100mcg (200 D) CFF,0.027111
3,B21 9RY,Metformin HCl_Tab 500mg,0.033294
4,B23 6DJ,Lansoprazole_Cap 30mg (E/C Gran),0.021384


In [40]:
grader.score.dw__items_by_region(result)

Your score:  1.0


## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [41]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

In [42]:
chem.shape

(3487, 2)

In [43]:
chem['CHEM SUB'].nunique()

3481

In [44]:

#chem['CHEM SUB'].value_counts()

2003         2
2113         2
1404000X0    2
0901011W0    2
1305030D0    2
            ..
1003020D0    1
0501070T0    1
140400040    1
0202080C0    1
0501014G0    1
Name: CHEM SUB, Length: 3481, dtype: int64

In [45]:
chem = chem.sort_values('CHEM SUB').drop_duplicates(subset = 'CHEM SUB', keep ='first')
chem.shape

(3481, 2)

In [46]:
#chem['CHEM SUB'].nunique()

3481

In [47]:
#chem ['NAME'].iloc[0]

'Other Antacid & Simeticone Preps'

In [48]:
#for opioid in opioids:
    #print(opioid in chem['NAME'].iloc[0])
    

False
False
False
False
False
False
False
False


In [49]:
for opioid in opioids:
    print(opioid in "codeine drug")

False
False
False
False
False
False
False
True


In [50]:
def is_opioid(drug_name, opioids):
    return any([opioid in drug_name.lower() for opioid in opioids])

In [51]:
chem['NAME'].apply(is_opioid, args =(opioids,)).sum()

35

In [52]:
re_opioids ="|". join(opioids)
print(re_opioids)

morphine|oxycodone|methadone|fentanyl|pethidine|buprenorphine|propoxyphene|codeine


In [53]:
chem['NAME'].apply(is_opioid, args =(opioids,)).sum()

35

In [54]:
chem['is_opioid'] = chem['NAME'].str.lower().str.contains(re_opioids)

In [55]:
chem.head()

,CHEM SUB,NAME,is_opioid
18,010101000,Other Antacid & Simeticone Preps,False
0,0101010A0,Alexitol Sodium,False
1,0101010B0,Almasilate,False
2,0101010C0,Aluminium Hydroxide,False
3,0101010D0,Aluminium Hydroxide With Magnesium,False


In [56]:
chem.shape

(3481, 3)

In [57]:
#scripts.merge(chem, left_on = 'bnf_code', right_on = 'CHEM SUB')

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,CHEM SUB,NAME,is_opioid
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,0106020C0,Bisacodyl,False
1,N81013,0106020C0,Bisacodyl_Tab E/C 5mg,19,28.10,28.14,860,0106020C0,Bisacodyl,False
2,N81029,0106020C0,Bisacodyl_Tab E/C 5mg,50,67.73,67.51,2074,0106020C0,Bisacodyl,False
3,N81029,0106020C0,Bisacodyl_Suppos 10mg,1,1.77,1.75,6,0106020C0,Bisacodyl,False
4,N81029,0106020C0,Dulcolax_Tab 5mg,1,3.78,3.61,56,0106020C0,Bisacodyl,False
5,N81062,0106020C0,Bisacodyl_Tab E/C 5mg,40,50.43,50.69,1544,0106020C0,Bisacodyl,False
6,N81062,0106020C0,Bisacodyl_Suppos 10mg,1,4.12,3.93,14,0106020C0,Bisacodyl,False
7,N81085,0106020C0,Bisacodyl_Tab E/C 5mg,35,43.42,43.83,1328,0106020C0,Bisacodyl,False
8,N81088,0106020C0,Bisacodyl_Tab E/C 5mg,67,97.10,95.08,2972,0106020C0,Bisacodyl,False
9,N81088,0106020C0,Bisacodyl_Suppos 5mg,1,4.16,3.86,20,0106020C0,Bisacodyl,False


These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table.. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

In [58]:
#scripts.merge(chem, left_on = 'bnf_code', right_on = 'CHEM SUB', how ='inner').shape


(854444, 10)

In [60]:
#scripts.merge(chem, left_on = 'bnf_code', right_on = 'CHEM SUB', how ='inner')['is_opioid'].sum()

34843

In [63]:
#scripts.merge(chem, left_on = 'bnf_code', right_on = 'CHEM SUB', how ='left').shape

(973193, 10)

In [67]:
scripts = (scripts.merge(chem, left_on = 'bnf_code', right_on = 'CHEM SUB', how ='left').fillna(False))

Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

In [68]:
opioids_per_practice = scripts.groupby('practice')['is_opioid'].mean()

In [69]:
opioids_per_practice

practice
A81005    0.033179
A81007    0.043329
A81011    0.046556
A81012    0.042793
A81017    0.038140
            ...   
Y05570    0.090909
Y05583    0.000000
Y05597    0.000000
Y05660    0.023055
Y05670    0.000000
Name: is_opioid, Length: 856, dtype: float64

How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

In [70]:
relative_opioids_per_practice = opioids_per_practice -scripts['is_opioid'].mean()

Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [72]:
scripts['practice'].value_counts()

N83028    2844
L83100    2797
D81043    2459
B81008    2396
B81026    2347
          ... 
Y03179       1
Y01174       1
Y03472       1
Y03010       1
Y05366       1
Name: practice, Length: 856, dtype: int64

In [74]:
scripts['is_opioid'].std()

0.18579817605238425

In [75]:
standard_error_per_practice = scripts['is_opioid'].std()/ (scripts['practice'].value_counts())**0.5
opioid_scores = relative_opioids_per_practice/ standard_error_per_practice

The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [76]:
unique_practices = practices.sort_values('name').drop_duplicates(subset = 'code', keep = 'first')
#anomalies = [("NATIONAL ENHANCED SERVICE", 11.6958178629, 7)] * 100

In [77]:
unique_practices= unique_practices.set_index('code')
unique_practices['z_score'] = opioid_scores
unique_practices['number'] = scripts['practice'].value_counts()

In [78]:
results =unique_practices.sort_values('z_score', ascending= False).head(100).loc[:, ['name', 'z_score', 'number']]

In [79]:
results.head()

,name,z_score,number
code,,,
Y01852,NATIONAL ENHANCED SERVICE,11.695818,7.0
Y03006,OUTREACH SERVICE NH / RH,7.339043,2.0
Y03668,BRISDOC HEALTHCARE SERVICES OOH,6.150582,60.0
G81703,H&R P C SPECIAL SCHEME,5.123032,36.0
Y04997,HMR BARDOC OOH,4.958866,321.0


In [80]:
grader.score.dw__script_anomalies(results)

Your score:  1.0


## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the percent growth in prescription rate from June 2016 to January 2017 for each `bnf_name`. We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative percent growth) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [6]:
scripts16 =pd.read_csv('dw-data/201606scripts_sample.csv.gz')

In [9]:
scripts16.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85638,0301011R0,Salamol_Inha 100mcg (200 D) CFF (Teva),2,2.92,2.73,2
1,N85638,0301011R0,Easyhaler_Salbutamol Sulf 200mcg (200D),1,6.63,6.15,1
2,N85638,0301020I0,Ipratrop Brom_Inh Soln 500mcg/2ml Ud,1,1.77,1.75,12
3,N85638,0301020I0,Ipratrop Brom_Inh Soln 250mcg/1ml Ud,1,4.47,4.15,20
4,N85638,0302000C0,Clenil Modulite_Inha 50mcg (200D),1,3.70,3.44,1


In [35]:
pct_growth = scripts['bnf_name'].value_counts() / scripts16['bnf_name'].value_counts() -1
pct_growth = pct_growth.rename('growth_rate')

In [36]:
a = pd.Series([1,2,3], index =['a','b','c'])
b = pd.Series([1,2,3], index =['b','c','d'])

In [38]:
a + b

a    NaN
b    3.0
c    5.0
d    NaN
dtype: float64

In [45]:
scripts.query('bnf_name == "365 Film 15cm x 20cm VP Adh Film Dress"')

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,CHEM SUB,NAME


In [48]:
result = pct_growth.to_frame()
result['raw_count'] = scripts16['bnf_name'].value_counts()
result.head()

,growth_rate,raw_count
365 Film 10cm x 12cm VP Adh Film Dress,NaN,NaN
365 Film 15cm x 20cm VP Adh Film Dress,NaN,1.0
365 Film 4cm x 5cm VP Adh Film Dress,NaN,1.0
365 Non Adherent 10cm x 10cm Pfa Plas Fa,0.0,3.0
365 Non Adherent 10cm x 20cm Pfa Plas Fa,NaN,NaN


In [49]:
result = result.query('raw_count >= 50')
result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3519 entries, 3m Health Care_Cavilon Durable Barrier C to Zumenon_Tab 1mg
Data columns (total 2 columns):
growth_rate    3485 non-null float64
raw_count      3519 non-null float64
dtypes: float64(2)
memory usage: 82.5+ KB


In [52]:
#result['growth_rate'] = result['growth_rate'].fillna(-1)
result = result.dropna()
result = result.sort_values('growth_rate', ascending= False)
result.head()

,growth_rate,raw_count
Butec_Transdermal Patch 5mcg/hr,3.467742,62.0
Butec_Transdermal Patch 10mcg/hr,3.000000,69.0
Fostair NEXThaler_Inh 200mcg/6mcg (120D),1.430233,86.0
Pneumococcal_Vac 0.5ml Vl (23 Valent),1.269430,193.0
Spiolto Respimat_Inha2.5/2.5mcg(60D)+Dev,1.269231,52.0


In [53]:
output = pd.concat([result.head(50), result.tail(50)]).reset_index().rename({'index': 'bnf_name'}, axis=1)
output

,bnf_name,growth_rate,raw_count
0,Butec_Transdermal Patch 5mcg/hr,3.467742,62.0
1,Butec_Transdermal Patch 10mcg/hr,3.000000,69.0
2,Fostair NEXThaler_Inh 200mcg/6mcg (120D),1.430233,86.0
3,Pneumococcal_Vac 0.5ml Vl (23 Valent),1.269430,193.0
4,Spiolto Respimat_Inha2.5/2.5mcg(60D)+Dev,1.269231,52.0
5,Trulicity_Inj 1.5mg/0.5ml Pf Pen,1.185185,54.0
6,CosmoCol_Paed Oral Pdr Sach 6.9g,1.177419,62.0
7,Dulaglutide_Inj 1.5mg/0.5ml Pf Dev,0.963415,82.0
8,ViATIM_Vac D/Chamber 160u/25mcg 1ml Pfs,0.912000,125.0
9,Empagliflozin_Tab 25mg,0.896000,125.0


In [ ]:
#script_growth = [("Butec_Transdermal Patch 5mcg\/hr", 3.4677419355, 62.0)] * 100

In [57]:
grader.score.dw__script_growth(output)

Your score:  1.0


## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [138]:
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
practices = pd.read_csv('dw-data/practices.csv.gz', names= col_names)
practices.head()
practices_unique = practices.sort_values('name').drop_duplicates(subset= 'code', keep = 'first')

In [143]:
p = 1/ scripts['bnf_code'].nunique()
rates = scripts['bnf_code'].value_counts() / scripts.shape[0]
rare_codes = rates < 0.1*p
scripts = scripts.set_index('bnf_code')
scripts['rare'] = rare_codes

Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [144]:
rare_codes.sum()

844

In [145]:
rare_cost_total = scripts.query('rare ==True').groupby('practice')['act_cost'].sum()
all_cost_total = scripts.groupby('practice')['act_cost'].sum()
all_cost_total


practice
A81005    103840.82
A81007    113482.49
A81011    159507.03
A81012     83296.81
A81017    232656.17
            ...    
Y05570        86.76
Y05583      1610.16
Y05597        90.41
Y05660      8762.51
Y05670        36.95
Name: act_cost, Length: 856, dtype: float64

In [146]:
rare_cost_prop = (rare_cost_total/all_cost_total).fillna(0)
rare_cost_prop


practice
A81005    0.012017
A81007    0.008381
A81011    0.005116
A81012    0.013747
A81017    0.007359
            ...   
Y05570    0.000000
Y05583    0.000000
Y05597    0.000000
Y05660    0.000323
Y05670    0.000000
Name: act_cost, Length: 856, dtype: float64

In [147]:
rare_cost_prop.isnull().sum()

0

Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [168]:
relative_rare_cost_prop = rare_cost_prop - scripts.query("rare == True")['act_cost'].sum()/scripts['act_cost'].sum()

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [169]:
standard_errors = relative_rare_cost_prop.std()

Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(post_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [170]:
rare_scores = relative_rare_cost_prop/standard_errors

In [171]:
rare_scores.isnull().sum()

0

In [172]:
practices_unique.head()

,name,addr_1,addr_2,borough,village,post_code,rare_score,rare_scores
code,,,,,,,,
P87657,(IRLAM) SALFORD CARE CTRS MEDICAL PRACTI,SALFORD CARE CTR(IRLAM),125 LIVERPOOL ROAD,IRLAM,MANCHESTER,M44 6DP,NaN,NaN
Y05381,0-19 EAST CHESHIRE HEALTH VISITORS,SALINAE CLINIC,LEWIN STREET,MIDDLEWICH,CHESHIRE,CW10 9FG,NaN,NaN
Y04082,0-19 PUBLIC HEALTH SERVICE HARTLEPOOL,CHATHAM CHILDRENS CENTRE,29 CHATHAM ROAD,HARTLEPOOL,NaN,TS24 8QG,NaN,NaN
Y00364,1 TO 1 CENTRE,BRENKLEY AVENUE,SHIREMOOR,NEWCASTLE UPON TYNE,NaN,NE27 0PR,NaN,NaN
P87652,1/LOWER BROUGHTON MEDICAL PRACTICE,LOWER BROUGHTON HTH.CTR.,GREAT CLOWES STREET,SALFORD,NaN,M7 1RD,-17.82855,-17.82855


In [173]:
practices_unique['rare_scores'] = rare_scores
practices_unique.head()

,name,addr_1,addr_2,borough,village,post_code,rare_score,rare_scores
code,,,,,,,,
P87657,(IRLAM) SALFORD CARE CTRS MEDICAL PRACTI,SALFORD CARE CTR(IRLAM),125 LIVERPOOL ROAD,IRLAM,MANCHESTER,M44 6DP,NaN,NaN
Y05381,0-19 EAST CHESHIRE HEALTH VISITORS,SALINAE CLINIC,LEWIN STREET,MIDDLEWICH,CHESHIRE,CW10 9FG,NaN,NaN
Y04082,0-19 PUBLIC HEALTH SERVICE HARTLEPOOL,CHATHAM CHILDRENS CENTRE,29 CHATHAM ROAD,HARTLEPOOL,NaN,TS24 8QG,NaN,NaN
Y00364,1 TO 1 CENTRE,BRENKLEY AVENUE,SHIREMOOR,NEWCASTLE UPON TYNE,NaN,NE27 0PR,NaN,NaN
P87652,1/LOWER BROUGHTON MEDICAL PRACTICE,LOWER BROUGHTON HTH.CTR.,GREAT CLOWES STREET,SALFORD,NaN,M7 1RD,-17.82855,-0.156754


In [174]:
rare_scripts = (practices_unique.reset_index()[['code', 'name', 'rare_scores']]
                .sort_values('rare_scores', ascending = False)
                .head(100))
rare_scripts

,code,name,rare_scores
2192,Y03472,CONSULTANT DIABETES TEAM,16.262687
2661,Y05320,DMC COMMUNITY DERMATOLOGY RBWF,15.128648
6789,Y04404,OUTPATIENTS JUBILEE HEALTH CENTRE,7.542139
2659,Y03484,DMC COMMUNITY DERMATOLOGY CLINIC,7.287222
2668,Y04424,DMC HEALTHCARE,6.838614
850,Y00631,BINGLEY DERMATOLOGY CLINIC,5.755998
581,A89038,BARMSTON MEDICAL CENTRE,5.664940
631,Y01696,BASSETLAW HOSPICE OF THE GOOD SHEPHERD,4.290685
6701,Y03699,OLDHAM DERMATOLOGY SERVICE,4.103744
10067,Y02045,VERNOVA HEALTHCARE CIC,3.180715


In [175]:
practices_unique['rare_scores'].isnull().sum()

9987

In [ ]:
rare_scripts = [("Y03472", "CONSULTANT DIABETES TEAM", 16.2626871247)] * 100

In [176]:
grader.score.dw__rare_scripts(rare_scripts)

Your score:  1.0


*Copyright &copy; 2020 The Data Incubator.  All rights reserved.*